In [9]:
import pandas as pd
path = '/home/dymiao/data/project/obey_rag/'
import torch
import jieba.analyse
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Read the .csv file
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import os
import re
from sentence_transformers import SentenceTransformer
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'
class SliceMethod:
    def __init__(self):
        self.end_char = ['。', '！', '？', '；', '.', '!', '?', ';']
    def any_char_slice(self,text):
        # 使用正则表达式根据中文句号、问号和感叹号分句
        sentences = re.split(r'(，|\,|。|！|\?|；|\.|\!|\?)', text)
        sentences = [s for s in sentences if s.strip()]  # 移除空句子
        return sentences
    def end_char_slice(self,text):
        # 使用正则表达式根据中文句号、问号和感叹号分句
        sentences = re.split(r'(。|！|\?|；|\.|\!|\?)', text)
        sentences = [s for s in sentences if s.strip()]  # 移除空句子
        return sentences
    # 固定长度切片
    def fixed_length_slice(self,text, length):
        return [text[i:i+length] for i in range(0, len(text), length)]
    
    # 滑动窗口切片
    def sliding_window_slice(self,text, window_size, step_size):
        return [text[i:i+window_size] for i in range(0, len(text), step_size)]

    # 基于段落切片
    def paragraph_slice(self,text):
        return text.split('\n')

class indexVector:
    def __init__(self):
        self.slice_tool = SliceMethod()
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        self.data_content = pd.read_csv('./t_cms_info_content_202405161305.csv')['TEXT']
    # 根据连续句子的语意进行连接
    def continue_sematic(self,article_index,delta = 0.5):
        sentence_vector = {}
        for index in article_index:
            text = self.data_content[index]
            meta_sentence = self.slice_tool.end_char_slice(text)
            meta_sentence_vector = self.model.encode(meta_sentence)
            current_sentence,current_vector = None,None
            i = 0 
            while i < len(meta_sentence):
                if current_sentence == None:
                    current_sentence = meta_sentence[i]
                    current_vector = meta_sentence_vector[i].reshape(1, -1)
                else:
                    if current_sentence[i] in self.slice_tool.end_char:
                        current_sentence += meta_sentence[i]
                    elif cosine_similarity(current_vector,meta_sentence_vector[i].reshape(1, -1)) > delta:
                        current_sentence += meta_sentence[i]
                        current_vector =0.5*current_vector +  0.5*meta_sentence_vector[i].reshape(1, -1)
                    else:
                        sentence_vector[current_vector] = current_sentence
                        current_sentence = meta_sentence[i]
                        current_vector = meta_sentence_vector[i]
                    i+=1
        return sentence_vector
    def consecutive_index(self,article_index,constant_length = 250):
        sentence_vector = {}
        for index in article_index:
            text = self.data_content[index]
            meta_sentence = []
            tmp_sentence = ""
            length = 0
            for sentence in self.slice_tool.any_char_slice(text):
                if len(tmp_sentence+sentence)<=constant_length:
                    tmp_sentence += sentence
                else:
                    meta_sentence.append(tmp_sentence)
                    tmp_sentence = sentence
                 
            meta_sentence.append(tmp_sentence)
            meta_sentence_vector = self.model.encode(meta_sentence)
            for i in range(len(meta_sentence)):
                if i !=0 and i != len(meta_sentence)-1:
                    sentence_vector[meta_sentence[i-1]+meta_sentence[i]+meta_sentence[i+1]] = meta_sentence_vector[i].reshape(1,-1)
                elif i==0:
                    sentence_vector[meta_sentence[i]+meta_sentence[i+1]] = meta_sentence_vector[i].reshape(1,-1)
                else:
                    sentence_vector[meta_sentence[i-1]+meta_sentence[i]] = meta_sentence_vector[i].reshape(1,-1)
        return sentence_vector 
    # 对句子进行聚类
    def cluster_sentences(self,article_index, num_clusters):
        meta_sentence = []
        for index in article_index:
            text = self.data_content[index]
            meta_sentence += self.slice_tool.end_char_slice(text)

        # 计算句子嵌入
        sentence_embeddings = self.model.encode(meta_sentence)
        # 归一化向量，使其范数为1，以便余弦相似度等同于欧氏距离
        normalized_embeddings = normalize(sentence_embeddings)
        # 使用 K-Means 算法进行聚类
        kmeans = KMeans(n_clusters=num_clusters, random_state=0)
        kmeans.fit(normalized_embeddings)
        # 获取聚类结果
        clusters = [[] for _ in range(num_clusters)]
        for i, label in enumerate(kmeans.labels_):
            clusters[label].append(meta_sentence[i])
        # 计算每个簇的平均向量
        cluster_centers = kmeans.cluster_centers_
        return clusters, cluster_centers        
class Retriver:
    # 初始化，读入数据以及数据预处理
    def __init__(self):
        self.data_info = pd.read_csv('./t_cms_info_202405160917.csv')['TITLE']
        self.data_content = pd.read_csv('./t_cms_info_content_202405161305.csv')['TEXT']
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        self.keywords = []
        for text in self.data_content:
            keys = jieba.analyse.extract_tags(text, topK=3, withWeight=True)
            self.keywords.append([key[0] for key in keys])
    def choose_article(self,title, top_n=3):
        # 编码标题
        title_vec = self.model.encode(title).reshape(1, -1)
        # 编码数据集中的标题
        titles_vec = np.array([self.model.encode(t).reshape(1, -1) for t in self.data_info])
        # 计算余弦相似度
        sim = cosine_similarity(title_vec, np.vstack(titles_vec))
        # 获取相似度最高的前 top_n 个索引
        top_indices = np.argsort(sim.flatten())[-top_n:][::-1]
        return top_indices
    
    def vector_similarity(self,question,embeding_group,delta = 0.1):
        v = self.model.encode(question).reshape(1, -1)
        return [key for key in embeding_group.keys() if cosine_similarity(v,embeding_group[key]) > delta]
    

In [10]:
slice = SliceMethod()
ix = indexVector()
rt = Retriver()
article_index = rt.choose_article('润滑油膜')
embedding_group = ix.consecutive_index(article_index)
rt.vector_similarity('我想买个润滑油膜',embedding_group)

['瓦楞机生产线的核心是瓦楞辊，其长期处于高温、重负荷及水汽存在的环境。故需要使用瓦楞机润滑脂对设备进行润滑和腐蚀保护。 市面上许多润滑脂以高滴点作为宣传手段，宣称耐温高达300℃或更高，但在实际使用过程中往往出现各种问题。润滑脂的使用不能仅关注温度一个指标，而需要综合考量才能长期使用。 客户的烦恼 客户主要生产瓦楞纸制品，生产线设备涉及BHS、京山轻机、广东万联等。 生产线图片 虽然不同的客户生产设备不同，但出现的问题大同小异。 主要出现的问题如下： 润滑脂析出，俗称“流油”。 润滑脂积炭，俗称“结焦”。 润滑脂润滑能力不够，导致设备磨损甚至损坏。 上述几点，轻则设备需要频繁加脂，增加人力成本；重则轴承卡死，增加维护成本，得不偿失。 瓦楞制品生产原理 瓦楞辊正常工作时，关键部位的运行温度在180℃以上，瞬间温度可达230℃以上。 因此瓦楞辊处的润滑脂长期处于高温状态，同时受到剪切作用，往往有稠度下降、易结焦的倾向。 瓦楞纸生产流程图 单瓦机生产原理 问题原因分析 1.润滑脂析出-流油 具体表现为：润滑脂稠度变稀，从轴承处流出。原因：润滑脂无法长期承受高温工况和剪切环境，',
 '瓦楞机生产线的核心是瓦楞辊，其长期处于高温、重负荷及水汽存在的环境。故需要使用瓦楞机润滑脂对设备进行润滑和腐蚀保护。 市面上许多润滑脂以高滴点作为宣传手段，宣称耐温高达300℃或更高，但在实际使用过程中往往出现各种问题。润滑脂的使用不能仅关注温度一个指标，而需要综合考量才能长期使用。 客户的烦恼 客户主要生产瓦楞纸制品，生产线设备涉及BHS、京山轻机、广东万联等。 生产线图片 虽然不同的客户生产设备不同，但出现的问题大同小异。 主要出现的问题如下： 润滑脂析出，俗称“流油”。 润滑脂积炭，俗称“结焦”。 润滑脂润滑能力不够，导致设备磨损甚至损坏。 上述几点，轻则设备需要频繁加脂，增加人力成本；重则轴承卡死，增加维护成本，得不偿失。 瓦楞制品生产原理 瓦楞辊正常工作时，关键部位的运行温度在180℃以上，瞬间温度可达230℃以上。 因此瓦楞辊处的润滑脂长期处于高温状态，同时受到剪切作用，往往有稠度下降、易结焦的倾向。 瓦楞纸生产流程图 单瓦机生产原理 问题原因分析 1.润滑脂析出-流油 具体表现为：润滑脂稠度变稀，从轴承处流出。原因：润滑脂无法长期承受高温工况和剪切环境，润滑脂中的皂基结构被破

: 